In [ ]:
import pandas as pd
import pandera.pandas as pa

from pandera.errors import SchemaErrors
from kedro.io.core import DatasetError

from src.preprocesamiento.preprocesamiento import limpieza
from src.preprocesamiento.pandera import build_schema
from src.config.config import load_catalog


In [3]:
widget = 'predict'

In [5]:
catalog, params = load_catalog(env="base")

if widget == 'train':
    print(f'Modo {widget}')
    # Validar que el nuevo dataset cumpla con el schema definido
    df_train = catalog.load('titanic_raw')
    try:
        schema = build_schema(params['data_validation'])
        validated_df = schema.validate(df_train,lazy=True)
        df_limpio = limpieza(df_train)
        catalog.save(data=df_limpio, ds_name='titanic_intermediate')

    except SchemaErrors as e:
        print("Errores encontrados en el schema")
        print(e.failure_cases)
        exit(1)
    except DatasetError as e:
        print("Error la data no se encuentra")
        print(e)
        exit(1)
    # Preprocesar el dataset
    
elif widget == 'predict':
    print(f'Modo {widget}')
    # Validar que el nuevo dataset cumpla con el schema definido
    try:
        schema = build_schema(params['data_validation'])
        df_predict = catalog.load('to_predict_raw')
        validated_df = schema.validate(df_predict,lazy=True)
        df_limpio = limpieza(df_predict)
        catalog.save(data=df_limpio, ds_name='to_predict_intermediate')

    except SchemaErrors as e:
        print("Errores encontrados en el schema")
        print(e.failure_cases)
        exit(1)

    except DatasetError as e:
        print("Error la data no se encuentra")
        print(e)
        exit(1)

elif widget == 'retraining':
    print(f'Modo {widget}')
    # Validar que el nuevo dataset cumpla con el schema definido
    try:
        schema = build_schema(params['data_validation'])
        df_predict = catalog.load('retraining')
        validated_df = schema.validate(df_predict,lazy=True)
        df_limpio = limpieza(df_predict) #---------------------limpiar o dejar de alguna forma----------------------------------------#
        catalog.save(data=df_limpio, ds_name='retraining')

    except SchemaErrors as e:
        print("Errores encontrados en el schema")
        print(e.failure_cases)
        exit(1)

    except DatasetError as e:
        print("Error la data no se encuentra")
        print(e)
        exit(1)
else:
    raise ValueError("Widget debe ser 'train' o 'retraining'")


Modo predict
